# Prompt Learning with NeMo

In this example, we utilize NeMo's [prompt learning](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/nemo_megatron/prompt_learning.html)
feature to showcase how to adapt a large language model (LLM) to 
a downstream task, such as financial sentiment predictions. 

The prompt learning technique shown in the example is [p-tuning](https://arxiv.org/abs/2103.10385), which adds a small prompt encoder network to the LLM
to produce virtual token embeddings that guide the model toward the desired output of the downstream task.

For more details on how to change hyperparameters for prompt learning in NeMo, see this [tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/nlp/Multitask_Prompt_and_PTuning.ipynb) which is also the basis for this NVFlare tutorial.

## Dependencies
We assume you followed the instructions [here](../../README.md#requirements) 
to install the NeMo framework and the NeMo-NVFlare package. 

## Download the pre-trained LLM
In this example, we use a `MegatronGPTModel`, a transformer-based language model based on the GPT architecture.

In [1]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
MegatronGPTModel.list_available_models()

[NeMo W 2023-05-03 21:53:35 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-05-03 21:53:35 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-05-03 21:53:37 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[PretrainedModelInfo(
 	pretrained_model_name=megatron_gpt_345m,
 	description=345M parameter GPT generative Megatron model.,
 	location=https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo
 )]

In [2]:
# Download the model from NGC
import os
model_file = "megatron_gpt_345m.nemo"
if not os.path.isfile(model_file):
    !wget "https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/$model_file"
else:
    print(f"{model_file} already downloaded.")

megatron_gpt_345m.nemo already downloaded.


## Data preprocessing
As our downstream task, we will use the [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for sentiment analysis.

The Financial PhraseBank dataset contains the sentiments for financial news headlines from a retail investor's perspective. Further details about the dataset can be found in Malo et al.'s ["Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts"](https://asistdl.onlinelibrary.wiley.com/doi/abs/10.1002/asi.23062).


#### 1. Download the preprocessing scripts
We use the preprocessing scripts provided by NeMo which can be downloaded from GitHub.

In [3]:
script_name = "prompt_learning_financial_phrase_bank_preprocessing.py"
if not os.path.isfile(script_name):
    !wget -N "https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/dataset_processing/nlp/financial_phrase_bank/$script_name"
else:
    print(f"{script_name} already downloaded.")

prompt_learning_financial_phrase_bank_preprocessing.py already downloaded.


#### 2. Download the Financial PhraseBank Dataset

Download the `FinancialPhraseBank-v1.0.zip` dataset from [here](https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v1.0/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v1.0.zip).

Then extract it under `./data`.

#### 3. Preprocess the dataset

In [4]:
!python3 prompt_learning_financial_phrase_bank_preprocessing.py

Saving train split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl
100%|███████████████████████████████████| 1811/1811 [00:00<00:00, 112901.27it/s]
Saving val split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_val.jsonl
100%|█████████████████████████████████████| 226/226 [00:00<00:00, 113468.12it/s]
Saving test split to data/FinancialPhraseBank-v1.0/financial_phrase_bank_test.jsonl
100%|█████████████████████████████████████| 227/227 [00:00<00:00, 122457.49it/s]


#### 4. Split the dataset to simulate clients
Next, we use three clients to simulate federated learning for p-tuning with NeMo.

In [5]:
!python3 data/split_financial_phrase_data.py --data_path data/FinancialPhraseBank-v1.0/financial_phrase_bank_train.jsonl --num_clients 3 --out_dir data/FinancialPhraseBank-v1.0_split

Loaded training data with 1811 entries
Save split 1 of 3 with 604 entries to data/FinancialPhraseBank-v1.0_split/site-1.jsonl
Save split 2 of 3 with 604 entries to data/FinancialPhraseBank-v1.0_split/site-2.jsonl
Save split 3 of 3 with 603 entries to data/FinancialPhraseBank-v1.0_split/site-3.jsonl


## Federated learning simulations
Next, we are using NVFlare's [simulator](https://nvflare.readthedocs.io/en/latest/user_guide/fl_simulator.html) to simulate each client training on their own dataset locally and all three clients training together using the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm implemented in NVFlare.

With this setting, we require a GPU with at least 16GB memory to run all clients in parallel on the same GPU. 
If you have multiple GPUs in your system, you can use the `gpu` argument to assign one GPU for each client, e.g., `gpu="0,1"`.

#### 1. Local P-Tuning
First, we modify the configuration files to include the current directory path to access the dataset and pre-trained LLM.

In [1]:
!python3 modify_configs.py --job_folder "jobs/gpt_p-tuning_local"

Set ROOT_DIR to /workspace/Code/nvflare/nemo_nvflare/integration/nemo/examples/prompt_learning


Next, simulate each client p-tuning on their local dataset using the FL simulator. To do this, we only run 1 round of FL, with each client running 50 p-tuning epochs on their local dataset.

In [ ]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/gpt_p-tuning_local",
    workspace="/tmp/nvflare/nemo/gpt_p-tuning_local",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-05-03 21:53:41,258 - SimulatorRunner - INFO - Create the Simulator Server.
2023-05-03 21:53:41,263 - Cell - INFO - server: creating listener on tcp://0:59755
2023-05-03 21:53:41,264 - Cell - INFO - server: created backbone external listener for tcp://0:59755
2023-05-03 21:53:41,265 - ConnectorManager - INFO - 25901: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-05-03 21:53:41,267 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:2793] is starting
2023-05-03 21:53:41,769 - Cell - INFO - server: created backbone internal listener for tcp://localhost:2793
2023-05-03 21:53:41,771 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:59755] is starting
2023-05-03 21:53:41,969 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 44111
2023-05-03 21:53:41,970 - SimulatorRunner - INFO - Deploy the Apps.
2023-05-03 21:53:41,980 - SimulatorRunner - INFO - Create the simulate 

[NeMo W 2023-05-03 21:53:59 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-05-03 21:53:59 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-05-03 21:53:59 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-05-03 21:53:59 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully

NEMO version 1.17.0
2023-05-03 21:54:00,880 - PromptLearner - INFO - [identity=site-3, run=simulate_job]: Initializing the Learner...
2023-05-03 21:54:00,880 - PromptLearner - INFO - [identity=site-3, run=simulate_job]: Using `MASTER_ADDR`: localhost and `MASTER_PORT`: None
2023-05-03 21:54:00,931 - PromptLearner - INFO - [identity=site-3, run=simulate_job]: Load model configuration from /tmp/nvflare/nemo/gpt_p-tuning_local/simulate_job/app_site-3/config/megatron_gpt_prompt_learning_config.yaml
2023-05-03 21:54:00,938 - PromptLearner - INFO - [identity=site-3, run=simulate_job]: Training with global_batch_size 8 and micro_batch_size 4
2023-05-03 21:54:00,942 - pytorch_lightning.utilities.rank_zero - INFO - Using 16bit None Automatic Mixed Precision (AMP)
2023-05-03 21:54:00,977 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2023-05-03 21:54:00,978 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2023-05-0

I0503 21:54:00.880515 140649257482048 fl_component.py:134] [identity=site-3, run=simulate_job]: Initializing the Learner...
I0503 21:54:00.880946 140649257482048 fl_component.py:134] [identity=site-3, run=simulate_job]: Using `MASTER_ADDR`: localhost and `MASTER_PORT`: None
I0503 21:54:00.931094 140649257482048 fl_component.py:134] [identity=site-3, run=simulate_job]: Load model configuration from /tmp/nvflare/nemo/gpt_p-tuning_local/simulate_job/app_site-3/config/megatron_gpt_prompt_learning_config.yaml
I0503 21:54:00.938791 140649257482048 fl_component.py:134] [identity=site-3, run=simulate_job]: Training with global_batch_size 8 and micro_batch_size 4
I0503 21:54:00.942872 140649257482048 accelerator_connector.py:758] Using 16bit None Automatic Mixed Precision (AMP)
I0503 21:54:00.977674 140649257482048 setup.py:163] GPU available: True (cuda), used: True
I0503 21:54:00.978206 140649257482048 setup.py:166] TPU available: False, using: 0 TPU cores
I0503 21:54:00.978397 14064925748204

2023-05-03 21:54:01,096 - PromptLearner - INFO - [identity=site-1, run=simulate_job]: Load model configuration from /tmp/nvflare/nemo/gpt_p-tuning_local/simulate_job/app_site-1/config/megatron_gpt_prompt_learning_config.yaml
2023-05-03 21:54:01,104 - PromptLearner - INFO - [identity=site-1, run=simulate_job]: Training with global_batch_size 8 and micro_batch_size 4
NEMO version 1.17.0
2023-05-03 21:54:01,111 - PromptLearner - INFO - [identity=site-2, run=simulate_job]: Initializing the Learner...
2023-05-03 21:54:01,112 - PromptLearner - INFO - [identity=site-2, run=simulate_job]: Using `MASTER_ADDR`: localhost and `MASTER_PORT`: None
2023-05-03 21:54:01,165 - PromptLearner - INFO - [identity=site-2, run=simulate_job]: Load model configuration from /tmp/nvflare/nemo/gpt_p-tuning_local/simulate_job/app_site-2/config/megatron_gpt_prompt_learning_config.yaml
2023-05-03 21:54:01,173 - PromptLearner - INFO - [identity=site-2, run=simulate_job]: Training with global_batch_size 8 and micro_ba

...

Epoch 34:   0%|          | 0/104 [00:00<?, ?it/s, loss=0.017, v_num=0, reduced_train_loss=0.000299, global_step=2549.0, val_loss=0.109]          

I0503 22:07:47.232909 140522389899072 fed_megatron_gpt_prompt_learning_model.py:99] val_loss: 0.10936349630355835


Epoch 33:  72%|███████▏  | 75/104 [00:17<00:06,  4.36it/s, loss=0.0527, v_num=0, reduced_train_loss=0.00284, global_step=2549.0, val_loss=0.144]]]  
Validation: 0it [00:00, ?it/s]
Epoch 34:   0%|          | 0/104 [00:00<?, ?it/s, loss=0.0527, v_num=0, reduced_train_loss=0.00284, global_step=2549.0, val_loss=0.125]          

I0503 22:07:54.600974 139755592410944 fed_megatron_gpt_prompt_learning_model.py:99] val_loss: 0.12539149820804596


Epoch 34:  33%|███▎      | 34/104 [00:08<00:17,  3.96it/s, loss=0.00691, v_num=0, reduced_train_loss=0.055, global_step=2583.0, val_loss=0.109]    

#### 2. Federated P-Tuning
We use the [FedAvg](https://arxiv.org/abs/1602.05629) algorithm to p-tune the model in a federated scenario. First, modify the configuration files again.

In [2]:
!python3 modify_configs.py --job_folder "jobs/gpt_p-tuning_fedavg"

Set ROOT_DIR to /workspace/Code/nvflare/nemo_nvflare/integration/nemo/examples/prompt_learning


Next, simulate the federated p-tuning using FedAvg. Here, each client p-tunes for one local epoch before sending their local model updates to the server for aggregation. This is repeated for 50 FL rounds.

In [ ]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/gpt_p-tuning_fedavg",
    workspace="/tmp/nvflare/nemo/gpt_p-tuning_fedavg",
    n_clients=3,
    threads=3
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

You can visualize the training process using TensorBoard

In [ ]:
!tensorboard --logdir /tmp/nvflare/nemo

## Results
In this scenario, all clients utilize the same validation set, allowing for a direct comparison between the locally p-tuned and federated global models. As anticipated, the FedAvg-trained global model exhibits lower validation loss than the models trained solely on their local datasets. This is because the global model has access to all client datasets and can, consequently, generalize better.

![validation loss](./val_loss.svg)

## Inference

We can use `model.generate()` to run inference after p-tuning the model. 
Let's define some test examples to feed to the p-tuned model to see its predictions.

In [ ]:
test_examples = [
    {"taskname": "sentiment", "sentence": "The products have a low salt and fat content ."},
    {"taskname": "sentiment", "sentence": "The agreement is valid for four years ."},
    {"taskname": "sentiment", "sentence": "Diluted EPS rose to EUR3 .68 from EUR0 .50 ."},
    {"taskname": "sentiment", "sentence": "The company is well positioned in Brazil and Uruguay ."},
    {"taskname": "sentiment", "sentence": "Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier ."},
]

Next, we will load the global model.

In [ ]:
import os
import torch
import pytorch_lightning as pl
from nemo_nvflare.fed_megatron_gpt_prompt_learning_model import FedMegatronGPTPromptLearningModel
from nemo_nvflare.utils import load_weights
from omegaconf import OmegaConf
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
from pytorch_lightning.plugins.environments import TorchElasticEnvironment

# Load model configuration used by one of the clients
config = OmegaConf.load("jobs/gpt_p-tuning_fedavg/app1/config/megatron_gpt_prompt_learning_config.yaml")

# Set GPT model path
config.model.language_model_path = "/workspace/Code/NeMo/upstream/tutorials/nlp/megatron_gpt_345m.nemo"

# Load task templates
config.model.task_templates = OmegaConf.load("jobs/gpt_p-tuning_fedavg/app1/config/task_templates.json")

# Set task that were learned
config.model.new_tasks = ["sentiment"]

# Setup cluster environment parameters
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'
strategy = NLPDDPStrategy(find_unused_parameters=False, no_ddp_communication_hook=True)
plugins = [TorchElasticEnvironment()]

# Set up the trainer and load the model that was used for p-tuning
trainer = pl.Trainer(plugins=plugins, strategy=strategy, **config.trainer)
model = FedMegatronGPTPromptLearningModel(cfg=config.model, trainer=trainer)
model.init_prompt_encoder()

print("Model initialized", type(model))

Overwrite the prompt encoder with the best global model

In [ ]:
ckpt = torch.load("/tmp/nvflare/nemo/gpt_p-tuning_fedavg/simulate_job/app_server/best_FL_global_model.pt")
global_weights = ckpt["model"]

n_loaded = load_weights(model, global_weights, device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
print(f"Loaded {n_loaded} of {len(global_weights)} weights")

Run the model

In [ ]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)

The expected output predictions look something like this

>      The products have a low salt and fat content . sentiment: neutral
>      ------------------------------
>      The agreement is valid for four years . sentiment: neutral
>      ------------------------------
>      Diluted EPS rose to EUR3 .68 from EUR0 .50 . sentiment: positive
>      ------------------------------
>      The company is well positioned in Brazil and Uruguay . sentiment: positive
>      ------------------------------
>      Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier . sentiment: negative
>      ------------------------------